In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch
import time
import sys
import resource
from torch.utils.data import DataLoader

from data_classes import *
from read_input import *
from read_trainset import *
from network import *
from prepare_batches import *
from traininit import *
from data_set import *
from data_loader import *
from optimization_step import *
from output_nn import *
from py_aeio import *

In [4]:
device = "cpu"

In [5]:
tin_file = "../../../BayesianNN/AenetBayesian/train.in"
tin = read_train_in(tin_file)
torch.manual_seed(tin.pytorch_seed)
np.random.seed(tin.numpy_seed)
if tin.verbose: io_input_reading(tin)
tin.train_file = '../../../BayesianNN/AenetBayesian/data.train.ascii'
tin.train_forces_file = '../../../BayesianNN/AenetBayesian/data.train.forces'

----------------------------------------------------------------------
                       Reading input information                      
----------------------------------------------------------------------

Reading input parameters.
These are the parameters selected for training:
        - TRAININGSET: data.train.ascii
        - TESTPERCENT: 0.1
        - ITERATIONS:  5000
        - ITERWRITE:   1
        - BATCHSIZE:   128
        - MEMORY_MODE: cpu

        - FORCES:      True
        - alpha:       0.1



In [6]:
if tin.verbose : io_trainingset_information()
list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)

----------------------------------------------------------------------
                   Reading training set information                   
----------------------------------------------------------------------

Training set information will be read now. If force training is
required this proccess may take some time.



In [7]:
N_removed = len(list_removed)
N_struc_E = len(list_structures_energy)
N_struc_F = len(list_structures_forces)
if tin.verbose : io_trainingset_information_done(tin, tin.trainset_params, N_struc_E, N_struc_F, N_removed)

if tin.verbose : io_prepare_batches()

N_batch_train, N_batch_valid = select_batch_size(tin, list_structures_energy, list_structures_forces)

# Join datasets with forces and only energies in a single torch dataset AND prepare batches
train_forces_data, valid_forces_data, train_energy_data, valid_energy_data = select_batches(tin, tin.trainset_params, device, list_structures_energy, list_structures_forces,
                                                                                        max_nnb, N_batch_train, N_batch_valid)

del list_structures_energy
del list_structures_forces

if tin.verbose : io_prepare_batches_done(tin, train_energy_data, train_forces_data)

The network output energy will be normalized to the interval [-1,1].
    Energy scaling factor:  f =     0.000502
    Atomic energy shift  :  s =  1985.644617


Number of structures in the data set:                 154
Number of structures with force information:           15

Atomic species in the training set: Pd  O

Average energy (eV/atom) :  1571.876188
Minimum energy (eV/atom) :    -4.525785
Maximum energy (eV/atom) :  3975.815020

----------------------------------------------------------------------
                    Preparing batches for training                    
----------------------------------------------------------------------

Batches for training are being prepared now. If force training is
required, this may take some time.

If the number of structures is not divisible by the batch size, the actual
batch size may be slightly changed.

Requested batch size: 128
Actual batch size   : 140
Number of batches   : 1

Energy batch size   : 126
Forces batch size   : 14



In [8]:
grouped_train_data = GroupedDataset(train_energy_data, train_forces_data,
									 memory_mode=tin.memory_mode, device=device, dataname="train")
grouped_valid_data = GroupedDataset(valid_energy_data, valid_forces_data,
									 memory_mode=tin.memory_mode, device=device, dataname="valid")

In [9]:
del train_forces_data
del valid_forces_data
del train_energy_data
del valid_energy_data

In [10]:
# 4. Initialize dataloader
grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)

In [11]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
    print('ciao')

grp_descrp[0] = grp_descrp[0].float()
grp_descrp[1] = grp_descrp[1].float()

logic_reduce[0] = logic_reduce[0].float()
logic_reduce[1] = logic_reduce[1].float()

ciao


In [35]:
from bayesian_network import BayesianNetAtoms
from pyro.nn.module import to_pyro_module_
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer.autoguide import AutoDiagonalNormal
import pyro

model = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)


to_pyro_module_(model)

weight_labels = [f'Pt_h{i}_weight' for i in range(3)] + [f'O_h{i}_weight' for i in range(3)]
bias_labels = [f'Pt_h{i}_bias' for i in range(3)] + [f'O_h{i}_bias' for i in range(3)]

i = 0
for m in model.modules():
    for name, value in list(m.named_parameters(recurse=False)):
        if name == 'weight':
            setattr(m, name, PyroSample(prior=dist.Normal(0, 20)
                                              .expand(value.shape)
                                              .to_event(value.dim())))
        if name == 'bias':
            setattr(m, name, PyroSample(prior=dist.Normal(0, 10)
                                              .expand(value.shape)
                                              .to_event(value.dim())))
            i += 1

def forward(model,  grp_descrp, logic_reduce):
    sigmas = {iesp : pyro.sample('noise_iesp', dist.Uniform(0,10)) for iesp in model.species}
    print('ciao')
    partial_E_ann = [0 for i in range(len(model.species))]
    for i, iesp in enumerate(model.species):
        with pyro.plate(f'data_{iesp}', len(grp_descrp[i])):
            mean = model.functions[iesp](grp_descrp[iesp])
            partial_E_ann[iesp] = pyro.sample(f'local_{iesp}', dist.Normal(mean, sigmas[iesp]))
    print(partial_E_ann)
    # Gather back all atoms corresponding to the same strucuture from partial_E_ann
    list_E_ann = torch.zeros( (len(logic_reduce[0])), device=model.device ).double()
    for iesp in range(len(model.species)):
        list_E_ann = list_E_ann + torch.einsum( "ij,ki->k", partial_E_ann[iesp], logic_reduce[iesp] )
    return list_E_ann

def bnn(grp_descrp, logic_reduce, grp_energy=None):
    # sigmas = {iesp : pyro.sample(f'noise_{iesp}', dist.Uniform(0,10)) for iesp in model.species}
    # partial_E_ann = [0 for i in range(len(model.species))]
    # for i, iesp in enumerate(model.species):
    #     with pyro.plate(f'data_{iesp}', len(grp_descrp[i])):
    #         mean = model.functions[i](grp_descrp[i])
    #         partial_E_ann[i] = pyro.sample(f'local_{iesp}', dist.Normal(mean, sigmas[iesp]))
    # print(partial_E_ann)
    # # Gather back all atoms corresponding to the same strucuture from partial_E_ann
    # list_E_ann = torch.zeros( (len(logic_reduce[0])), device=model.device ).double()
    # for iesp in range(len(model.species)):
    #     list_E_ann = list_E_ann + torch.einsum( "ij,ki->k", partial_E_ann[iesp], logic_reduce[iesp] )
    
    partial_E_ann = [0 for i in range(len(model.species))]
    for iesp in range(len(model.species)):
        partial_E_ann[iesp] = model.functions[iesp](grp_descrp[iesp])

    # Gather back all atoms corresponding to the same strucuture from partial_E_ann
    
    sigma = pyro.sample('noise', dist.Uniform(0,10))
    with pyro.plate('data', len(logic_reduce[0])):
        list_E_ann = torch.zeros( (len(logic_reduce[0])), device=model.device ).double()
        for iesp in range(len(model.species)):
            list_E_ann = list_E_ann + torch.einsum( "ij,ki->k", partial_E_ann[iesp], logic_reduce[iesp] )
        pyro.sample('obs', dist.Normal(list_E_ann, sigma), obs=grp_energy)
    return list_E_ann

# model = BayesianNetAtoms(net)       
guide = AutoDiagonalNormal(bnn)

guide

AutoDiagonalNormal()

In [36]:
model.functions[0]

PyroSequential(
  (Linear_Sp1_F1): PyroLinear(in_features=30, out_features=5, bias=True)
  (Active_Sp1_F1): PyroTanh()
  (Linear_Sp1_F2): PyroLinear(in_features=5, out_features=5, bias=True)
  (Active_Sp1_F2): PyroTanh()
  (Linear_Sp1_F3): PyroLinear(in_features=5, out_features=1, bias=True)
)

In [46]:
from pyro.infer import SVI, Trace_ELBO

pyro.clear_param_store()
adam = pyro.optim.Adam({"lr": 0.05})
svi = SVI(bnn, guide, adam, loss=Trace_ELBO())
for x in range(10000):
    if x % 1000:
        print(svi.step(grp_descrp, logic_reduce, grp_energy))

26772.509490729855
23985.795355451875
24890.46382087948
19434.546266581878
22646.60000382669
19671.320174467895
19527.54377269704
15594.622574556653
15479.471078794042
21002.20309952988
13607.685493784878
17123.15160884134
14538.703296216294
13308.84702172141
14785.332847191978
15433.906114546082
11638.107438262305
10539.987807847472
10961.067435963261
8941.58302404174
10008.791525314506
9473.419108233213
8768.481191252606
9529.02401076832
9435.94519977626
8832.791400924061
8707.959163790983
8623.7520785904
8390.941765136078
8466.825441609224
8334.096314137856
7572.201825521295
8731.432656573284
7740.240201389392
7768.709770649097
7502.666144688865
8113.320256184925
8177.918630583884
7140.655572433801
7717.302925161119
8385.482427048164
7226.418889758317
8014.825917654944
7843.4051851952845
7143.996386277013
6430.338132435462
6475.48335995566
7598.837702657589
6644.023871505528
6660.885319304467
6254.682661224117
6079.915623832083
6719.521629541639
6880.14304037364
6046.7571822976815
5

KeyboardInterrupt: 

In [41]:
from pyro.infer import Predictive

def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0).numpy(),
            "std": torch.std(v, 0).numpy(),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0].numpy(),
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0].numpy(),
        }
    return site_stats


predictive = Predictive(bnn, guide=guide, num_samples=800,
                        return_sites=("weight", "obs", "_RETURN"))

samples = predictive(grp_descrp, logic_reduce)
mu = summary(samples)

In [45]:
for i in zip(mu['obs']['mean'], mu['obs']['std'],grp_energy):
    print(i)

(-137.61800862784443, 112.6388750345242, tensor(107.8656, dtype=torch.float64))
(100.21418507106956, 55.45179789325323, tensor(-62.9981, dtype=torch.float64))
(497.08553335213475, 19.276646674096977, tensor(107.8850, dtype=torch.float64))
(22.86188822619847, 128.77837874199923, tensor(107.8829, dtype=torch.float64))
(-20.087458726350548, 15.417987577895735, tensor(107.9120, dtype=torch.float64))
(141.45160941604544, 17.74170552340541, tensor(107.8639, dtype=torch.float64))
(121.10633655737655, 19.22804272889547, tensor(107.8646, dtype=torch.float64))
(122.08770781372932, 15.963303277162831, tensor(107.8906, dtype=torch.float64))
(-153.34454194089193, 20.021523496364168, tensor(-99.9961, dtype=torch.float64))
(-624.0373936802899, 20.214673630703224, tensor(-99.9691, dtype=torch.float64))
(140.5344836673001, 23.236376090859963, tensor(107.8644, dtype=torch.float64))
(-206.14651182411382, 39.45596621079678, tensor(107.8754, dtype=torch.float64))
(163.60058633749566, 19.758942396481718, te

In [ ]:
model = BayesianNetAtoms(net.functions, net.species, net.device)

pyro.render_model(model, model_args=([torch.ones(108, 52),torch.ones(108, 52)], [torch.ones([4, 108]),torch.ones([4, 108])], torch.ones(4,)))#,grp_energy))

NameError: name 'net' is not defined

In [ ]:
# 4. Initialize dataloader
grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)

In [ ]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]

In [ ]:
grp_energy

tensor([-27.0000, -17.8317,   2.4570,   3.1589], dtype=torch.float64)

In [ ]:
logic_reduce[0].shape

torch.Size([4, 108])

In [ ]:
grp_descrp[0] = grp_descrp[0].float()
logic_reduce[0] =logic_reduce[0].float()

In [ ]:
model.forward(grp_descrp, logic_reduce, grp_energy)

tensor([ 302.2620, -196.0373,  209.5111,  309.6779], grad_fn=<AddBackward0>)

In [ ]:
model.svi.step(grp_descrp, logic_reduce,grp_energy)

NotImplementedError: 
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
for m in model.modules():
    m.model(data_batch[0][10][0], data_batch[0][10][0], data_batch[0][12][0])

NotImplementedError: 

In [ ]:
model.model(data_batch[0][10], data_batch[0][12])

RuntimeError: expected scalar type Float but found Double

In [ ]:
for data_batch in grouped_train_loader:
    grp_descrp, grp_energy, logic_reduce, grp_N_atom = data_batch[0][10], data_batch[0][11], data_batch[0][12], data_batch[0][14]


In [ ]:
grp_N_atom

tensor([27., 27., 27., 27.], dtype=torch.float64)

In [ ]:
grp_descrp[0].shape

torch.Size([108, 52])

In [ ]:
for data_batch in grouped_train_loader:
    model.step(data_batch[0][10], data_batch[0][12])

RuntimeError: expected scalar type Float but found Double
                              Trace Shapes:        
                               Param Sites:        
                              Sample Sites:        
model.functions.0.Linear_Sp1_F1.weight dist | 15 52
                                      value | 15 52
  model.functions.0.Linear_Sp1_F1.bias dist | 15   
                                      value | 15   
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
data_batch[0][10]

[tensor([[-0.0789,  0.7212, -1.5706,  ...,  1.3901,  1.2308,  0.2393],
         [-1.0732,  0.9951,  0.0450,  ...,  0.4152, -0.1812,  0.9815],
         [ 0.3986, -0.4489,  0.1622,  ..., -0.2772, -0.7212,  0.4384],
         ...,
         [-0.6189, -2.1412,  1.5223,  ..., -1.7583, -1.2087, -2.4926],
         [ 1.6111,  0.4451, -0.4222,  ...,  0.3068,  0.3127, -0.5332],
         [-0.1283, -0.6618, -0.9102,  ...,  0.5214, -0.0743,  0.6787]],
        dtype=torch.float64),
 tensor([], size=(0, 52), dtype=torch.float64)]